Projeto Sistema de Monitoramento de Avanços no Campo da Genômica


DÚVIDA

-> Tem diferença (qual melhor) entre usar o requests e o cliente library ? https://newsapi.org/docs/client-libraries/python

1. Consumo de dados com a News API:

Implementar um mecanismo para consumir dados de notícias de fontes confiáveis e especializadas em genômica e medicina personalizada, a partir da News API:
https://newsapi.org/


In [0]:
import requests
from pyspark.sql.functions import *
from delta.tables import *

API_KEY = '5220c4535529484381b6e56b7f95b0aa'

In [0]:
#Fazer request dos dados

s_tema = "diabetes"
s_from = "2024-09-18"
s_sortBy = "publishedAt" # "publishedAt" or "popularity" or "relevancy"

url = ('https://newsapi.org/v2/everything?'
       f'q={s_tema}&'
       f'from={s_from}&'
       f'sortBy={s_sortBy}&'
       f'apiKey={API_KEY}')

print(url)
response = requests.get(url)

https://newsapi.org/v2/everything?q=diabetes&from=2024-09-18&sortBy=publishedAt&apiKey=5220c4535529484381b6e56b7f95b0aa


In [0]:
artigos = response.json()
print(artigos)

{'status': 'ok', 'totalResults': 5547, 'articles': [{'source': {'id': None, 'name': 'The BMJ'}, 'author': "Barrett, S., Begg, S., Dunford, A., O'Halloran, P., Rodda, K., Denniss, E., Breckon, J., Kingsley, M.", 'title': 'An expert consensus on the most effective components of integrated motivational interviewing and cognitive behavioural therapy for lifestyle behaviour change: protocol for an online modified Delphi study', 'description': 'Introduction Behaviour change interventions are advocated as a key approach to manage behavioural risk factors such as insufficient physical activity, poor diet and smoking. Integrated motivational interviewing and cognitive behavioural therapy (MI-CBT) inter…', 'url': 'https://bmjopen.bmj.com/content/14/10/e088988', 'urlToImage': 'https://bmjopen.bmj.com/pages/wp-content/uploads/sites/7/2019/07/bmjopen-default-cover.png', 'publishedAt': '2024-10-17T22:28:58Z', 'content': 'Background\r\nChronic diseases such as diabetes, heart disease and some cancers

In [0]:
print(artigos["articles"][0].keys())

dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])


In [0]:
#gravar em um df spark
lista_dados = []

for artigo in artigos["articles"]:
  artigo["source"] = artigo["source"]["name"]
  lista_dados.append(artigo)

df = spark.createDataFrame(lista_dados)

In [0]:
display(df)

author,content,description,publishedAt,source,title,url,urlToImage
"Barrett, S., Begg, S., Dunford, A., O'Halloran, P., Rodda, K., Denniss, E., Breckon, J., Kingsley, M.","Background Chronic diseases such as diabetes, heart disease and some cancers are leading causes of ill health and morbidity worldwide.1 2 Modifiable risk factors such as insufficient physical activi… [+6484 chars]","Introduction Behaviour change interventions are advocated as a key approach to manage behavioural risk factors such as insufficient physical activity, poor diet and smoking. Integrated motivational interviewing and cognitive behavioural therapy (MI-CBT) inter…",2024-10-17T22:28:58Z,The BMJ,An expert consensus on the most effective components of integrated motivational interviewing and cognitive behavioural therapy for lifestyle behaviour change: protocol for an online modified Delphi study,https://bmjopen.bmj.com/content/14/10/e088988,https://bmjopen.bmj.com/pages/wp-content/uploads/sites/7/2019/07/bmjopen-default-cover.png
"Wen, S., Zhao, X., Lin, X., Fu, Z., Qin, S., Pan, Q., Liu, F., He, W., Liu, T.","Introduction In gastro-oesophageal reflux disease (GERD), gastroduodenal contents reflux into the oesophagus, causing symptoms such as acid reflux, heartburn, regurgitation and post-sternal pain and… [+7898 chars]",Background Gastro-oesophageal reflux disease (GERD) is a highly prevalent disorder worldwide and developing effective treatment strategies for GERD is a clinical priority. GERD is associated with anxiety and depression. Several approaches have been develope…,2024-10-17T22:28:58Z,The BMJ,Acupoint catgut embedding advantage in treating gastro-oesophageal reflux disease (ACE-GERD): study protocol for a randomised controlled trial,https://bmjopen.bmj.com/content/14/10/e081059,https://bmjopen.bmj.com/pages/wp-content/uploads/sites/7/2019/07/bmjopen-default-cover.png
"Alam, M. T., Veettil, S. T., Krishnan, J. I., Alsalmi, R. G., Jacob, J., Mousa, A. T., Sakallah, S. A. M., Mahmood, S., Al Nuaimi, A.",STRENGTHS AND LIMITATIONS OF THIS STUDY The study uses a cross-sectional design to provide a snapshot of the primary healthcare systems response to implementing home delivery services for me… [+16265 chars],"Objectives During the COVID-19 epidemic, due to the high risk associated with travel and group gatherings, the Ministry of Public Health in Qatar introduced home delivery of pharmaceutical services. This study aims to investigate the satisfaction and preferen…",2024-10-17T22:28:58Z,The BMJ,Satisfaction and preference for home delivery pharmaceutical services among patients attending primary care settings in Qatar during COVID-19 pandemic: a cross-sectional study,https://bmjopen.bmj.com/content/14/10/e082842,https://bmjopen.bmj.com/pages/wp-content/uploads/sites/7/2019/07/bmjopen-default-cover.png
Leandro Costa Criscuolo,"Uma nova insulina desenvolvida pela Novo Nordisk, que se adapta automaticamente aos níveis de glicose no sangue, mostrou eficácia em reduzir o risco de hipoglicemia em animais. O estudo desta criação… [+1496 chars]","Abordagem reduz o risco de hipoglicemia, oferecendo mais segurança aos diabéticos ao ajustar automaticamente a absorção de glicose no sangue O post Novo ‘interruptor’ que ativa insulina pode ajudar no tratamento da diabetes apareceu primeiro em Olhar Digital.",2024-10-17T22:22:08Z,Olhardigital.com.br,Novo ‘interruptor’ que ativa insulina pode ajudar no tratamento da diabetes,https://olhardigital.com.br/2024/10/17/medicina-e-saude/novo-interruptor-que-ativa-insulina-pode-ajudar-no-tratamento-da-diabetes/,https://olhardigital.com.br/wp-content/uploads/2024/10/insulina.jpg
El Universal (México),"Los alimentos que elegimos a diario para nuestra dieta deben priorizar lograr una completa nutrición para el organismo. Una dieta adecuada, balanceada y combinada con una actividad física regular, es… [+1725 chars]",Aporta vitaminas del grupo B y es fácil de hallar para sumar a la dieta; todo lo que hay que saber; mirá el v

In [0]:
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- description: string (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- source: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- urlToImage: string (nullable = true)



In [0]:
#Realizar os filtros necessários

In [0]:
#Salvar em delta
df.write.format("delta").save("/FileStore/diabetes")

In [0]:
#Ler arquivos delta
delta_table = spark.read.format("delta").load("/FileStore/diabetes")
display(delta_table)

author,content,description,publishedAt,source,title,url,urlToImage
Cnn,"Democratic presidential nominee, Vice President Kamala Harris pauses while speaking during a campaign rally at the Rawhide Event Center on October 10, in Chandler, Arizona. Brandon Bell/Getty Images… [+3322 chars]","“She possesses the physical and mental resiliency required to successfully execute the duties of the Presidency,” her doctor, US Army physician Joshua R. Simmons, said in a letter summarizing her",2024-10-13T03:44:12Z,WPSD Local 6,"Vice President Harris is in ‘excellent health,’ according to detailed letter from her physician",https://www.wpsdlocal6.com/news/vice-president-harris-is-in-excellent-health-according-to-detailed-letter-from-her-physician/article_7b20e548-88bc-11ef-83f1-2736a5726ff9.html,https://bloximages.newyork1.vip.townnews.com/wpsdlocal6.com/content/tncms/assets/v3/editorial/7/b2/7b20e548-88bc-11ef-83f1-2736a5726ff9/670aad7358573.preview.jpg?crop=1763%2C926%2C0%2C124&resize=1200%2C630&order=crop%2Cresize
Lois Balado,"El 28 % de fármacos con receta que se dispensan en España son medicamentos que actúan sobre el sistema nervioso central, el mayor porcentaje de todos los grupos anatómicos que atiende la sanidad. Sol… [+17559 chars]","En un escenario socioeconómico de total incertidumbre y ante la falta de alternativas públicas que permitan el acceso a la psicoterapia, el consumo de recetas de antidepresivos inhibidores selectivos de recaptación de la serotonina sigue disparado",2024-10-13T03:00:00Z,Lavozdegalicia.es,"Antideprimidos para siempre, cuando la pastilla se eterniza: «Llevo más de 30 años medicada»",https://www.lavozdegalicia.es/noticia/lavozdelasalud/salud-mental/2024/10/12/antideprimidos-siempre-pastilla-eterniza-llevo-30-anos-medicada/00031728720241859157709.htm,https://cflvdg.avoz.es/sc/8ZWglxauRBROjf_anyFQPjgGW5A=/1280x/2024/10/11/00121728645024933680295/Foto/ANTIDEPRIMIDOS.png
Carmen G. Mariñas,"David Baker, Geoffrey E. Hinton y Victor Ambros son tres de los galardonados con un premio Nobel científico este 2024. Son, además, hombres anglosajones. Si le sumamos el provenir de una familia de e… [+8306 chars]","Una investigación destaca que en 122 años de historia, solo un 4 % de las personas premiadas en categorías científicas fueron mujeres",2024-10-13T03:00:00Z,Lavozdegalicia.es,"Hombre, de familia rica y estadounidense o europeo: los premios Nobel tienen un perfil marcado",https://www.lavozdegalicia.es/noticia/sociedad/2024/10/11/hombre-familia-rica-estadounidense-europeo-premios-nobel-perfil-marcado/00031728654836802451280.htm,https://img.lavdg.com/sc/-XfgqKRo5g_ucR3SFtoq9iRWeZo=/1280x/2024/10/11/00121728663542694665666/Foto/reu_20241008_192617869.jpg
R. ROMAR,Fue algo histórico. Por primer vez un avance científico logrado gracias a la inteligencia artificial ha sido reconocido con un premio Nobel. Y muy probablemente no será la última. La herramienta de I… [+8448 chars],El premio muestra el enorme potencial de la IA en el avance científico,2024-10-13T03:00:00Z,Lavozdegalicia.es,"Una aplicación de inteligencia artificial gana el Nobel, pero no será la última vez",https://www.lavozdegalicia.es/noticia/sociedad/2024/10/13/aplicacion-inteligencia-artificial-gana-nobel-sera-ultima-vez/0003_202410G13P34991.htm,https://img.lavdg.com/sc/O-gPVFHU-Od73e6vZaypzoLTZxU=/1280x/2024/10/11/00121728669961639894152/Foto/reu_20241010_074435592.jpg
Thatyana Costa,"O que é colesterol alto e quais os riscos? - iStock/HYWARDS O colesterol alto é uma condição que afeta muitas pessoas e, muitas vezes, não apresenta sintomas visíveis. Mesmo assim, ele aumenta signi… [+1782 chars]","O colesterol alto é uma condição que afeta muitas pessoas e, muitas vezes, não apresenta sintomas visíveis. Mesmo assim, ele aumenta significativamente os",2024-10-13T02:01:02Z,Catracalivre.com.br,Dicas essenciais para baixar o colesterol alto e evitar riscos,https://catracalivre.com.br/saude-bem-estar/dicas-essenciais-para-baixar-o-colesterol-alto-e-evitar-ris

In [0]:
# Carregar a tabela Delta com notícias previamente armazenadas
delta_table = DeltaTable.forPath(spark, "/FileStore/diabetes")
display(delta_table)

In [0]:
#Merge de arquivos não duplicados

# Carregar a tabela Delta com notícias previamente armazenadas
delta_table = DeltaTable.forPath(spark, "/FileStore/diabetes")

# Criar DataFrame com novas notícias
novas_noticias_df = spark.createDataFrame(novas_noticias)

# Usar o MERGE para inserir apenas as notícias que ainda não existem
delta_table.alias("old").merge(
    novas_noticias_df.alias("new"),
    "old.titulo = new.titulo AND old.data_publicacao = new.data_publicacao"
).whenNotMatchedInsertAll().execute()


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2664458781476944>:7
      4 delta_table = DeltaTable.forPath(spark, "/FileStore/diabetes")
      6 # Criar DataFrame com novas notícias
----> 7 novas_noticias_df = spark.createDataFrame(novas_noticias)
      9 # Usar o MERGE para inserir apenas as notícias que ainda não existem
     10 delta_table.alias("old").merge(
     11     novas_noticias_df.alias("new"),
     12     "old.titulo = new.titulo AND old.data_publicacao = new.data_publicacao"
     13 ).whenNotMatchedInsertAll().execute()

NameError: name 'novas_noticias' is not defined

In [0]:
#Salvar em parquet [DESCONSIDERAR]
#dbutils.fs.rm("genoma.parquet", True)
#df.write.parquet('/FileStore/genoma.parquet')
#display(spark.read.parquet('/FileStore/genoma.parquet'))

3. Cargas em Batches:
Armazenar as notícias relevantes em um formato estruturado e facilmente acessível para consultas e análises posteriores. Essa carga deve acontecer 1 vez por hora. Se as notícias extraídas já tiverem sidos armazenadas na carga anterior, o processo deve ignorar e não armazenar as notícias novamente, os dados carregados não podem ficar duplicados.
